In [0]:
# ## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

# copyfile(src="../input/input2/train_unlabeld_dataloader.p", dst="../working/train_unlabelled_dataloader.p")
# copyfile(src="../input/input2/train_labeled_dataloader.p", dst="../working/train_labelled_dataloader.p")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Capstone_Hyperparam_Tuning

/content/drive/My Drive/Capstone_Hyperparam_Tuning


In [0]:
#!pip install transformers

In [27]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from generate_dataloaders import *
from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/content/drive/My Drive/Capstone_Hyperparam_Tuning/evaluation.py'>

## Get Dataloaders

In [0]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [0]:
path = os.getcwd()
data_dir = path + '/data/'

In [0]:
train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [0]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [13]:
print(torch.__version__)

1.3.1


## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [0]:
class neuralNetBow(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, opts):
        super(neuralNetBow, self).__init__()
        self.embed = nn.Embedding(opts['vocab_size'], opts['emb_dim'], padding_idx=0)
        self.upweight = opts['upweight']
        self.lambda_loss = opts['lambda_loss']
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
        
        # upweight by flagged_index
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff

In [0]:
class KMeansCriterion(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = cluster_distances.sum()
        return loss, cluster_assignments

In [0]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Dataloader stuff

In [0]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)

    return tokens, labels, flagged_indices, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)

    return tokens, labels, flagged_indices, train_loader_unlabelled_iter

## Training Function

In [0]:
def train_model(model, centroids, criterion, optimizer, train_loader_labelled, train_loader_unlabelled, valid_loader, num_epochs=10, num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)
    lambda_loss = model.lambda_loss

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in tqdm(range(num_epochs)):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in range(num_batches):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
            
            total_batch_loss = cluster_loss_unlabelled + lambda_loss * cluster_loss_labelled
        
            # run update step
            optimizer.zero_grad()
            total_batch_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_labelled, sentence_embed_labelled)
    
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_unlabelled, sentence_embed_unlabelled)

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            sub_folder = "/"
            path_to_save+=sub_folder
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts') #change options depending on model inputs required
        
    return model, centroids, train_losses, val_losses

In [20]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'
print("Current Device:",current_device)

Current Device: cuda


## Hyperparameter Tuning Loop

In [0]:
def get_save_directory(opts):
    path = os.getcwd()
    model_folder = 'baseline_semisupervised_randomized_embeddings/'
    model_dir = path + '/models/' + model_folder
    
    # subfolder for each hyperparam config
    emb_dim = opts['emb_dim']
    upweight = opts['upweight']
    lambda_loss = opts['lambda_loss']
    subfolder = "emb_dim="+str(emb_dim) + ",upweight="+str(upweight) + ",lambda="+str(lambda_loss) + '/'
    
    # need to actually create these subfolders lol
    try:
        os.makedirs(model_dir + subfolder) # will throw error if subfolder already exists
    except:
        pass
    
    return model_dir + subfolder

In [0]:
def train_config(opts):
    model = neuralNetBow(opts).to(current_device)
    centroids = centroid_init(2, opts['emb_dim'],train_loader_labelled, model, current_device)
    criterion = KMeansCriterion().to(current_device)
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    train_model(model, centroids, criterion, optimizer, train_loader_labelled, train_loader_unlabelled, val_loader, num_epochs=10, path_to_save=path_to_save)

In [0]:
emb_dims = [128, 256, 512]
upweights = [1, 5, 10, 25]
lambda_losses = [.1, .5, 1, 5, 10, 25]

for emb_dim in emb_dims:
    for upweight in upweights:
        for lambda_loss in lambda_losses:
            opts = {
                'vocab_size': 20000,
                'emb_dim': emb_dim,
                'upweight': upweight,
                'lambda_loss': lambda_loss
            }
            train_config(opts)

/content/drive/My Drive/2019 Fall/Capstone group/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=0.1/


2019-12-08 20:39:14.965918 | Epoch 0
Average training loss at batch  0 : 0.729
Average training loss after epoch  0 : 0.117
Average validation loss after epoch  0 : 0.126
2019-12-08 20:39:20.263463 | Epoch 1
Average training loss at batch  0 : 0.063
Average training loss after epoch  1 : 0.026
Average validation loss after epoch  1 : 0.062
2019-12-08 20:39:25.465990 | Epoch 2
Average training loss at batch  0 : 0.029
Average training loss after epoch  2 : 0.015
Average validation loss after epoch  2 : 0.041
2019-12-08 20:39:30.666392 | Epoch 3
Average training loss at batch  0 : 0.027
Average training loss after epoch  3 : 0.008
Average validation loss after epoch  3 : 0.031
2019-12-08 20:39:35.855142 | Epoch 4
Average training loss at batch  0 : 0.013
Average training loss after epoch  4 : 0.007
Average validation loss after epoch  4 : 0.027
2019-12-08 20:39:40.992418 | Epoch 5
Average training loss at batch  0 : 0.011
Average training loss after epoch  5 : 0.006
Average validation lo

2019-12-08 20:40:07.448457 | Epoch 0
Average training loss at batch  0 : 1.009
Average training loss after epoch  0 : 0.140
Average validation loss after epoch  0 : 0.112
2019-12-08 20:40:12.602315 | Epoch 1
Average training loss at batch  0 : 0.056
Average training loss after epoch  1 : 0.026
Average validation loss after epoch  1 : 0.056
2019-12-08 20:40:17.814703 | Epoch 2
Average training loss at batch  0 : 0.035
Average training loss after epoch  2 : 0.015
Average validation loss after epoch  2 : 0.038
2019-12-08 20:40:23.059512 | Epoch 3
Average training loss at batch  0 : 0.022
Average training loss after epoch  3 : 0.008
Average validation loss after epoch  3 : 0.031
2019-12-08 20:40:28.191307 | Epoch 4
Average training loss at batch  0 : 0.021
Average training loss after epoch  4 : 0.006
Average validation loss after epoch  4 : 0.026
2019-12-08 20:40:33.305890 | Epoch 5
Average training loss at batch  0 : 0.005
Average training loss after epoch  5 : 0.005
Average validation lo

2019-12-08 20:41:00.091342 | Epoch 0
Average training loss at batch  0 : 1.461
Average training loss after epoch  0 : 0.175
Average validation loss after epoch  0 : 0.110
2019-12-08 20:41:05.321611 | Epoch 1
Average training loss at batch  0 : 0.062
Average training loss after epoch  1 : 0.028
Average validation loss after epoch  1 : 0.058
2019-12-08 20:41:10.477299 | Epoch 2
Average training loss at batch  0 : 0.026
Average training loss after epoch  2 : 0.015
Average validation loss after epoch  2 : 0.040
2019-12-08 20:41:15.676528 | Epoch 3
Average training loss at batch  0 : 0.022
Average training loss after epoch  3 : 0.009
Average validation loss after epoch  3 : 0.031
2019-12-08 20:41:21.046506 | Epoch 4
Average training loss at batch  0 : 0.008
Average training loss after epoch  4 : 0.007
Average validation loss after epoch  4 : 0.027
2019-12-08 20:41:26.227505 | Epoch 5
Average training loss at batch  0 : 0.012
Average training loss after epoch  5 : 0.006
Average validation lo

2019-12-08 20:41:52.678603 | Epoch 0
Average training loss at batch  0 : 4.652
Average training loss after epoch  0 : 0.549
Average validation loss after epoch  0 : 0.145
2019-12-08 20:41:57.854714 | Epoch 1
Average training loss at batch  0 : 0.211
Average training loss after epoch  1 : 0.063
Average validation loss after epoch  1 : 0.079
2019-12-08 20:42:03.084433 | Epoch 2
Average training loss at batch  0 : 0.080
Average training loss after epoch  2 : 0.029
Average validation loss after epoch  2 : 0.055
2019-12-08 20:42:08.302654 | Epoch 3
Average training loss at batch  0 : 0.037
Average training loss after epoch  3 : 0.016
Average validation loss after epoch  3 : 0.044
2019-12-08 20:42:13.500516 | Epoch 4
Average training loss at batch  0 : 0.027
Average training loss after epoch  4 : 0.012
Average validation loss after epoch  4 : 0.036
2019-12-08 20:42:18.773764 | Epoch 5
Average training loss at batch  0 : 0.013
Average training loss after epoch  5 : 0.009
Average validation lo

...

2019-12-08 21:47:13.079645 | Epoch 0
Average training loss at batch  0 : 68.791
Average training loss after epoch  0 : 5.619
Average validation loss after epoch  0 : 2.259
2019-12-08 21:47:20.245617 | Epoch 1
Average training loss at batch  0 : 0.964
Average training loss after epoch  1 : 0.514
Average validation loss after epoch  1 : 0.896
2019-12-08 21:47:27.270385 | Epoch 2
Average training loss at batch  0 : 0.543
Average training loss after epoch  2 : 0.230
Average validation loss after epoch  2 : 0.580
2019-12-08 21:47:34.346889 | Epoch 3
Average training loss at batch  0 : 0.437
Average training loss after epoch  3 : 0.117
Average validation loss after epoch  3 : 0.461
2019-12-08 21:47:41.807525 | Epoch 4
Average training loss at batch  0 : 0.295
Average training loss after epoch  4 : 0.091
Average validation loss after epoch  4 : 0.347
2019-12-08 21:47:48.879661 | Epoch 5
Average training loss at batch  0 : 0.568
Average training loss after epoch  5 : 0.069
Average validation l

2019-12-08 21:48:25.312188 | Epoch 0
Average training loss at batch  0 : 97.729
Average training loss after epoch  0 : 7.464
Average validation loss after epoch  0 : 2.219
2019-12-08 21:48:32.355264 | Epoch 1
Average training loss at batch  0 : 1.193
Average training loss after epoch  1 : 0.576
Average validation loss after epoch  1 : 1.090
2019-12-08 21:48:39.376288 | Epoch 2
Average training loss at batch  0 : 0.850
Average training loss after epoch  2 : 0.266
Average validation loss after epoch  2 : 0.610
2019-12-08 21:48:46.448073 | Epoch 3
Average training loss at batch  0 : 0.187
Average training loss after epoch  3 : 0.149
Average validation loss after epoch  3 : 0.493
2019-12-08 21:48:53.922588 | Epoch 4
Average training loss at batch  0 : 1.531
Average training loss after epoch  4 : 0.100
Average validation loss after epoch  4 : 0.393
2019-12-08 21:49:00.975462 | Epoch 5
Average training loss at batch  0 : 0.083
Average training loss after epoch  5 : 0.076
Average validation l

2019-12-08 21:49:37.253959 | Epoch 0
Average training loss at batch  0 : 273.493
Average training loss after epoch  0 : 23.965
Average validation loss after epoch  0 : 2.628
2019-12-08 21:49:44.389578 | Epoch 1
Average training loss at batch  0 : 5.119
Average training loss after epoch  1 : 1.625
Average validation loss after epoch  1 : 1.130
2019-12-08 21:49:51.553971 | Epoch 2
Average training loss at batch  0 : 0.723
Average training loss after epoch  2 : 0.575
Average validation loss after epoch  2 : 0.693
2019-12-08 21:49:58.639416 | Epoch 3
Average training loss at batch  0 : 0.343
Average training loss after epoch  3 : 0.293
Average validation loss after epoch  3 : 0.493
2019-12-08 21:50:06.157603 | Epoch 4
Average training loss at batch  0 : 0.333
Average training loss after epoch  4 : 0.189
Average validation loss after epoch  4 : 0.415
2019-12-08 21:50:13.131387 | Epoch 5
Average training loss at batch  0 : 0.176
Average training loss after epoch  5 : 0.137
Average validation

2019-12-08 21:50:49.465038 | Epoch 0
Average training loss at batch  0 : 536.989
Average training loss after epoch  0 : 45.078
Average validation loss after epoch  0 : 2.735
2019-12-08 21:50:56.458773 | Epoch 1
Average training loss at batch  0 : 9.734
Average training loss after epoch  1 : 2.970
Average validation loss after epoch  1 : 1.173
2019-12-08 21:51:03.603432 | Epoch 2
Average training loss at batch  0 : 4.141
Average training loss after epoch  2 : 0.980
Average validation loss after epoch  2 : 0.760
2019-12-08 21:51:10.670275 | Epoch 3
Average training loss at batch  0 : 0.780
Average training loss after epoch  3 : 0.472
Average validation loss after epoch  3 : 0.624
2019-12-08 21:51:18.089168 | Epoch 4
Average training loss at batch  0 : 0.436
Average training loss after epoch  4 : 0.310
Average validation loss after epoch  4 : 0.491
2019-12-08 21:51:25.178461 | Epoch 5
Average training loss at batch  0 : 0.404
Average training loss after epoch  5 : 0.215
Average validation

2019-12-08 21:52:01.071026 | Epoch 0
Average training loss at batch  0 : 1235.215
Average training loss after epoch  0 : 109.099
Average validation loss after epoch  0 : 2.997
2019-12-08 21:52:08.168984 | Epoch 1
Average training loss at batch  0 : 27.999
Average training loss after epoch  1 : 7.304
Average validation loss after epoch  1 : 1.252
2019-12-08 21:52:15.188235 | Epoch 2
Average training loss at batch  0 : 6.287
Average training loss after epoch  2 : 2.285
Average validation loss after epoch  2 : 0.824
2019-12-08 21:52:22.328090 | Epoch 3
Average training loss at batch  0 : 2.184
Average training loss after epoch  3 : 1.079
Average validation loss after epoch  3 : 0.618
2019-12-08 21:52:29.774453 | Epoch 4
Average training loss at batch  0 : 1.748
Average training loss after epoch  4 : 0.658
Average validation loss after epoch  4 : 0.515
2019-12-08 21:52:36.811025 | Epoch 5
Average training loss at batch  0 : 1.063
Average training loss after epoch  5 : 0.476
Average validat

## Evaluation

In [0]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [0]:
def evaluate_config(opts,verbose=True):
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    model = neuralNetBow(opts) #change according to model inputs
    model.load_state_dict(torch.load(path_to_save+'model_dict.pt',map_location=lambda storage, loc: storage))
    model = model.to(current_device)
    criterion = KMeansCriterion().to(current_device)
    centroids = torch.load(path_to_save+'centroids',map_location=lambda storage, loc: storage).to(current_device)
    
    TP_cluster, FP_cluster, results_dict = evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device, verbose)
    results_dict.update(opts)
    return TP_cluster, FP_cluster, results_dict

In [30]:
emb_dims = [128, 256, 512]
upweights = [1, 5, 10, 25]
lambda_losses = [.1, .5, 1, 5, 10, 25]

results_df = pd.DataFrame()
for emb_dim in emb_dims:
    for upweight in upweights:
        for lambda_loss in lambda_losses:
            opts = {
                'vocab_size': 20000,
                'emb_dim': emb_dim,
                'upweight': upweight,
                'lambda_loss': lambda_loss
            }
            _, _, results_dict = evaluate_config(opts,False)
            results_df = results_df.append(results_dict,ignore_index=True)
        
results_df = results_df[['emb_dim','upweight','lambda_loss','Accuracy','F1 score','Precision','Recall',
                        'TP_rate','FP_rate','FN_rate','TN_rate']]

/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=0.1/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=0.5/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=1/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=5/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=10/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=1,lambda=25/
/content/drive/My Drive/Capstone_Hyperparam_Tuning/models/baseline_semisupervised_randomized_embeddings/emb_dim=128,upweight=5,lambda=0.1/
/content/drive/My Drive/Capstone_

In [31]:
results_df

,emb_dim,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
0,128.0,1.0,0.1,0.545498,0.662205,0.890995,0.526906,0.890995,0.109005,0.800000,0.200000
1,128.0,1.0,0.5,0.539934,0.658493,0.887097,0.523569,0.887097,0.112903,0.807229,0.192771
2,128.0,1.0,1.0,0.566194,0.674357,0.898345,0.539773,0.898345,0.101655,0.765957,0.234043
3,128.0,1.0,5.0,0.613658,0.704655,0.921760,0.570324,0.921760,0.078240,0.694444,0.305556
4,128.0,1.0,10.0,0.613476,0.705724,0.926952,0.569748,0.926952,0.073048,0.700000,0.300000
...,...,...,...,...,...,...,...,...,...,...,...
67,512.0,25.0,0.5,0.612289,0.710232,0.950292,0.566998,0.950292,0.049708,0.725714,0.274286
68,512.0,25.0,1.0,0.723441,0.773950,0.946882,0.654429,0.946882,0.053118,0.500000,0.500000
69,512.0,25.0,5.0,0.678932,0.743577,0.931034,0.618954,0.931034,0.068966,0.573171,0.426829
70,512.0,25.0,10.0,0.663765,0.734864,0.931925,0.606595,0.931925,0.068075,0.604396,0.395604


In [32]:
results_df.sort_values(['F1 score'],ascending=False).head(10)

,emb_dim,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
20,128.0,25.0,1.0,0.737844,0.782190,0.941441,0.669020,0.941441,0.058559,0.465753,0.534247
62,512.0,10.0,1.0,0.732966,0.778283,0.937360,0.665366,0.937360,0.062640,0.471429,0.528571
44,256.0,25.0,1.0,0.730588,0.777454,0.941176,0.662252,0.941176,0.058824,0.480000,0.520000
14,128.0,10.0,1.0,0.730964,0.775562,0.929670,0.665280,0.929670,0.070330,0.467742,0.532258
55,512.0,5.0,0.5,0.722658,0.774203,0.950935,0.652867,0.950935,0.049065,0.505618,0.494382
68,512.0,25.0,1.0,0.723441,0.773950,0.946882,0.654429,0.946882,0.053118,0.500000,0.500000
38,256.0,10.0,1.0,0.723208,0.770878,0.931264,0.657620,0.931264,0.068736,0.484848,0.515152
32,256.0,5.0,1.0,0.715479,0.765918,0.930958,0.650584,0.930958,0.069042,0.500000,0.500000
7,128.0,5.0,0.5,0.711675,0.764601,0.936508,0.646016,0.936508,0.063492,0.513158,0.486842
37,256.0,10.0,0.5,0.705725,0.763927,0.952267,0.637786,0.952267,0.047733,0.540816,0.459184
